# Text Classification

Using SVM

In [ ]:
import pickle
from pathlib import Path
import os

import pandas as pd
import numpy as np
from sklearn import metrics

from nlp.nb import NBClassifier

### Loading articles

In [ ]:
STORAGE_PATH = './storage/nb.model'

good_articles = pd.read_table('./articles/good.articles', sep='\n', names=['body'])
good_articles['label'] = 0 # good
bad_articles  = pd.read_table('./articles/bad.articles', sep='\n', names=['body'])
bad_articles['label'] = 1 # bad

# joining
articles = pd.concat([good_articles, bad_articles], ignore_index=True)

# persistently shuffling
index_path = f"{STORAGE_PATH}.shuffled_index"
if Path(index_path).is_file():
  file_size = os.path.getsize(index_path)
  print(f"Loading index from stored file ({file_size} bytes)...")
  with open(index_path, 'rb') as fp:
    shuffled_index = pickle.load(fp)
else:
  print('Shuffling index for the first time ...')
  shuffled_index = np.random.permutation(articles.index)
  print('Saving index on disk for further access...')
  with open(index_path, 'wb') as fp:
    pickle.dump(shuffled_index, fp)
  file_size = os.path.getsize(index_path)
  print(f"Done. It took {file_size} bytes on the disk.")

articles = articles.reindex(shuffled_index)

print(f"Counts:\n{articles['label'].value_counts()}")
articles.head()

### Splitting between train and test

In [ ]:
test_first_index = int(articles.shape[0] * 0.35)

train_data = articles[:test_first_index]
test_data  = articles[test_first_index:]

print(train_data.shape)
print(test_data.shape)

### Training

### Predicting

In [ ]:
report = metrics.classification_report(test_data['label'], predictions)
print(report)

### Results
